In [83]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as sn
import single_photons.utils.constants as ct
from single_photons.environment import Particle

In [84]:
def compute_gamma(radius, pressure, rho=2200, T = 293, index_refraction=1.444):
    m_p = 4 / 3 * np.pi * radius**3 * rho
    v_gas = np.sqrt(3 * ct.kb * T / ct.m_gas)
    gamma = 15.8 * radius**2 * pressure / (m_p * v_gas)
    return gamma

def compute_omega(power, waist, rho=2200, index_refraction=1.444):
    omega = (
    np.sqrt(12 / np.pi)
    * np.sqrt((index_refraction**2 - 1) / (index_refraction**2 + 2)) ** 3
    * np.sqrt(tweezer_power)
    / (tweezer_waist**2 * np.sqrt(rho * ct.c))
    )
    return omega

def compute_scattered_power(power, waist, wavelength, rho=2200, index_refraction=1.444):
    Nm = rho/(ct.amu*60.08) #SiO2 molecular density
    k_tweezer = 2*np.pi/wavelength
    pol_permit_ratio = 3/Nm*(index_refraction**-1)/(index_refraction**2+2) #from C-M
    sigma = (8*np.pi/3)*(pol_permit_ratio*k_tweezer*k_tweezer/(4*np.pi*8.85e-12))**2    
    I0 = 2*P/waist
    return I0*sigma

def compute_backaction(wavelength, p_scat, A=0.71):
    ba_force = np.sqrt(2*(A**2 + 0.4)*ct.hbar*2*np.pi*p_scat/(ct.c*wavelength))
    return ba_force

def compute_ideal_detection(wavelength, p_scat):
    return np.sqrt(2*ct.hbar*ct.c/((A**2+0.4)*4*(2*np.pi/wavelength)*p_scat))

def compute_parameters_simulation(power, wavelength, waist, radius, pressure, fs, eta_detection,
                                  rho=2200, index_refraction=1.444, T=293):
    gamma = compute_gamma(radius, pressure, rho=rho, T=T, index_refraction=index_refraction)
    omega = compute_omega(power, waist, rho=rho, index_refraction=index_refraction)
    p_scat = compute_scattered_power(power, waist, wavelength, rho=rho, index_refraction=index_refraction)
    ba_force = compute_backaction(wavelength, p_scat)
    std_z = compute_ideal_detection(wavelength, p_scat)
    std_detection = std_z*np.sqrt(fs/eta_detection)
    return gamma, omega, ba_force, std_detection

In [85]:
p = 1e-6
R = 147e-9
wavelength = 1.06e-6
power = 300e-3
waist = 0.6e-6
eta_detection = 0.3
fs = 1/30e-9

In [87]:
gamma, omega, ba_force, std_detection = compute_parameters_simulation(power, wavelength, waist, 
                                                                      R, p, fs, eta_detection)
gamma, omega, ba_force**2, std_detection**2

(2.321754696705159e-05,
 501266.40589032974,
 1.6768421844819337e-39,
 7.369142389804251e-22)

In [90]:
ba_force = np.sqrt(2*(A**2 + 0.4)*ct.hbar*2*np.pi*p_scattered/(ct.c*tweezer_wavelength))

In [93]:
coupling = ba_force**2/(4*np.pi)
coupling

1.334388612226558e-40

In [92]:
std_z = np.sqrt(2*ct.hbar*ct.c/((A**2+0.4)*4*(2*np.pi/tweezer_wavelength)*p_scattered))

In [43]:
std_detection = std_z*np.sqrt(fs/eta_detection)

In [33]:
coupling = 6.68e-42 
env = Particle(omega, gamma, coupling, eta_detection=0.174, T=273)

In [39]:
np.power(env.backaction_std*env.zp_p, 2)

8.394335570391926e-41

In [89]:
std_z = np.sqrt(2*ct.hbar*ct.c/((A**2+0.4)*4*k_tweezer*22.4e-6))
std_z

1.1145795945588402e-14

In [47]:
std_z*env.backaction_std*env.zp_p, ct.hbar

(1.0533557373057139e-34, 1.054571e-34)

In [50]:
std_z/(np.sqrt(0.347))

0.010295802359945929

In [54]:
(std_z**2)*fs/2

2.2029901394229897e-21

In [ ]:
eta_detection=0.347